In [12]:
import numpy as np
import pandas as pd
import time

import requests
import bs4
import json
import re

In [13]:
def stock_prices(ticker):
    """
    ticker is the abbreviated symbol for a stock e.g.AAPL
    this function returns the daily price history of the requested stock as a dataframe
    """
    stock_endpoint = 'https://financialmodelingprep.com/api/v3/historical-price-full/'
    response = requests.get(stock_endpoint + ticker + '?apikey=70407133ea11d7284c70bbca4eee2547').json()
    type(response) == dict
    return pd.DataFrame(response['historical'])

def ny_times_articles(keyword):
    url_dict = {}
    index = 0
    for i in range(10):
        response = requests.get("https://api.nytimes.com/svc/search/v2/articlesearch.json?q=apple&fq=news_desk:Business&page="+str(i)+"&api-key=fO0tDSRQQdU68GkuXbMjt1uA2FYImzVp").json()
        try:
            docs = response['response']['docs']
            for item in docs:
                if item['web_url'][8] == 'w': # To remove 'https://bits.blogs.nytimes...' or ''https://dealbook.nytimes...'
                    link = item['web_url']
                    date = link[24:28] + '-' + link[29:31] + '-' + link[32:34]
                    url_dict[index] = tuple((link, date))
                    index += 1
        except KeyError:
            continue
    return url_dict

In [14]:
test_dict = ny_times_articles("Apple")

test_dict

{0: ('https://www.nytimes.com/2020/12/16/technology/facebook-takes-the-gloves-off-in-feud-with-apple.html',
  '2020-12-16'),
 1: ('https://www.nytimes.com/2020/12/13/business/media/apple-gawker-tim-cook.html',
  '2020-12-13'),
 2: ('https://www.nytimes.com/2020/12/23/business/dealbook/trump-stimulus-veto.html',
  '2020-12-23'),
 3: ('https://www.nytimes.com/2020/12/01/technology/amazon-apple-chips-intel-arm.html',
  '2020-12-01'),
 4: ('https://www.nytimes.com/2020/12/17/technology/google-antitrust-monopoly.html',
  '2020-12-17'),
 5: ('https://www.nytimes.com/2020/12/17/business/dealbook/tech-apple-facebook-fight.html',
  '2020-12-17'),
 6: ('https://www.nytimes.com/2020/12/15/technology/big-tech-regulation-europe.html',
  '2020-12-15'),
 7: ('https://www.nytimes.com/2020/12/14/technology/big-tech-lobbying-europe.html',
  '2020-12-14'),
 8: ('https://www.nytimes.com/2020/12/09/technology/personaltech/amazon-halo-review.html',
  '2020-12-09'),
 9: ('https://www.nytimes.com/2020/11/18/t

In [17]:
stocks = stock_prices('AAPL')

In [21]:
# use the closing prices of the stock as the daily prices and the differences in each day as the percentage rise or fall
# the difference of stock prices between date1 and date2 will be stored in the row of date1 for ease of comparison

prices = stocks['close']
dates = stocks['date']
difference = stocks['close'].astype('float64').diff()
pct_change = difference / prices

stock = pd.DataFrame({'date': dates, 'price':prices, 'change': difference, 'pct_change': pct_change})
stock

,date,price,change,pct_change
0,2020-12-31,132.690002,NaN,NaN
1,2020-12-30,133.720001,1.029999,0.007703
2,2020-12-29,134.869995,1.149994,0.008527
3,2020-12-28,136.690002,1.820007,0.013315
4,2020-12-24,131.970001,-4.720001,-0.035766
...,...,...,...,...
1254,2016-01-08,24.240000,-0.392500,-0.016192
1255,2016-01-07,24.112499,-0.127501,-0.005288
1256,2016-01-06,25.174999,1.062500,0.042205
1257,2016-01-05,25.677500,0.502501,0.019570


In [6]:
text = []
title = []
abstract = []

headers = {'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'}
article = requests.get('https://www.nytimes.com/2006/07/01/business/01online.html', headers = headers)
soup = bs4.BeautifulSoup(article.content, 'html.parser')
article_text_p = soup.find_all('p', attrs={'class': 'css-axufdj evys1bk0'})
abstract_text_p = soup.find('p', attrs={'class': 'css-w6ymp8 e1wiw3jv0'})
title_text_h1 = soup.find('h1', attrs={'data-test-id': 'headline'})
temp = []
print(title_text_h1)
title.append(title_text_h1.text)
abstract.append(abstract_text_p.text)
    
for item in article_text_p:
    temp.append(item.text)
space = ' '
article_text = space.join(temp)
text.append(article_text)

<h1 class="css-1fyocly e1h9rw200" data-test-id="headline" id="link-2698b7df">Apple's Got a Secret</h1>


AttributeError: 'NoneType' object has no attribute 'text'